# Import Libraries

In [1]:
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim

# Data loading and Preprocessing

In [2]:

# Define transforms for data preprocessing
transform = transforms.Compose([
    transforms.Grayscale(),  # Convert to grayscale
    transforms.Resize((128, 128)),  # Resize to 128x128
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize
])


# Define the path to the root folder of your labeled image dataset
root_folder = 'D:\\ezi_project\\X-Ray project'

# Create the ImageFolder dataset for the training set
train_dataset = ImageFolder(root=root_folder + '/train', transform=transform)

# Create the ImageFolder dataset for the testing set
test_dataset = ImageFolder(root=root_folder + '/test', transform=transform)


In [3]:

# Define batch size for training and testing
batch_size = 48

# Create the train data loader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Create the test data loader
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


# Building model

In [4]:

# Define the CNN model
class BoneFractureCNN(nn.Module):
    def __init__(self):
        super(BoneFractureCNN, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        
        self.fc1 = nn.Linear(32 * 32 * 32, 128)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(128, 2)  # 2 classes: fracture and non-fracture
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        
        x = x.view(x.size(0), -1)
        
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.fc2(x)
        
        return x

# Define hyperparameters
learning_rate = 0.001
num_epochs = 2

# Instantiate the CNN model
model = BoneFractureCNN()

# Define loss function and optimizer
error = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)



# Training Model

In [5]:
# Training loop
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images
        labels = labels
        
        # Forward pass
        outputs = model(images)
        loss = error(outputs, labels)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
         # Print training progress
        if (i+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{total_step}], Loss: {loss.item():.4f}')

Epoch [1/2], Step [10/185], Loss: 0.6710
Epoch [1/2], Step [20/185], Loss: 0.6845
Epoch [1/2], Step [30/185], Loss: 0.6812
Epoch [1/2], Step [40/185], Loss: 0.6346
Epoch [1/2], Step [50/185], Loss: 0.6509
Epoch [1/2], Step [60/185], Loss: 0.5928
Epoch [1/2], Step [70/185], Loss: 0.5586
Epoch [1/2], Step [80/185], Loss: 0.5416
Epoch [1/2], Step [90/185], Loss: 0.5733
Epoch [1/2], Step [100/185], Loss: 0.5343
Epoch [1/2], Step [110/185], Loss: 0.5508
Epoch [1/2], Step [120/185], Loss: 0.5823
Epoch [1/2], Step [130/185], Loss: 0.5701
Epoch [1/2], Step [140/185], Loss: 0.4302
Epoch [1/2], Step [150/185], Loss: 0.5834
Epoch [1/2], Step [160/185], Loss: 0.5612
Epoch [1/2], Step [170/185], Loss: 0.4966
Epoch [1/2], Step [180/185], Loss: 0.5050
Epoch [2/2], Step [10/185], Loss: 0.2831
Epoch [2/2], Step [20/185], Loss: 0.3663
Epoch [2/2], Step [30/185], Loss: 0.2714
Epoch [2/2], Step [40/185], Loss: 0.4028
Epoch [2/2], Step [50/185], Loss: 0.3588
Epoch [2/2], Step [60/185], Loss: 0.2757
Epoch [

# Model Evaluation

In [6]:

model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    
    for images, labels in test_loader:
        images = images
        labels = labels
        
        outputs = model(images)
        _,predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy:.2f}%')


Test Accuracy: 76.17%
